# Find precipitation value for pulsed and upslope storms divided into 5 wind regimes

In [1]:
import sys
#sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')
sys.path.append('/home/franzihe/Documents/Python/Thesis')

import numpy as np
import netCDF4
import matplotlib.pyplot as plt

import createFolder as cF
np.warnings.filterwarnings('ignore')

import seaborn as sns
import fct_windspeed_sorting as fct

In [2]:
# plot cosmetics

sns.set_context('paper')

sns.set(font = 'Serif', font_scale = 1.2, )
sns.set_style('ticks',
                  {'font.family':'serif',
                   'grid.linestyle': '--'           
                  },
                   )

In [3]:
m = [ '12', '01', '02']
station = 'Haukeliseter'
meps_run = ['CTRL', 'XCCR']

In [4]:
savefig = 0
if savefig == 1:
    figdir = '../../../Figures/Weathermast_MEPS_Retrieval/%s/CTRL_XCCR/' %(station)
    cF.createFolder('%s/' %figdir)
    form = 'png'

In [5]:
f = dict()
obs_precip_diff      = dict()
obs_precip_diff_2deg = dict()
obs_west_idx         = dict()
obs_east_idx         = dict()
obs_wind_dir         = dict()
obs_wind_speed       = dict()
obs_precip_total_2deg= dict()
obs_precip_west_2deg = dict()
obs_precip_east_2deg = dict()
    
meps_precip_diff     = dict()
meps_precip_diff_2deg= dict()
meps_west_idx        = dict()
meps_east_idx        = dict()
meps_wind_dir        = dict()
meps_wind_speed      = dict()
meps_precip_total_2deg= dict()
meps_precip_west_2deg = dict()
meps_precip_east_2deg = dict()
    
ret_precip_diff      = dict()
ret_precip_total     = dict()
ret_precip_west      = dict()
ret_precip_east      = dict()



for meps in meps_run:
    meps_wind_dir[meps]         = dict()
    meps_wind_speed[meps]       = dict()
    meps_precip_diff[meps]      = dict()
    meps_precip_diff_2deg[meps] = dict()
    meps_west_idx[meps]         = dict()
    meps_east_idx[meps]         = dict()
    meps_precip_total_2deg[meps]= dict()
    meps_precip_west_2deg[meps] = dict()
    meps_precip_east_2deg[meps] = dict()
    
    date = []
    for month in m:
        if month == '12' or  month == '01':
            t = np.arange(1,32)
        if month == '02':
            t = np.arange(1,29)
        if month == '11' or month == '12':
            year = '2016'
        if month == '01' or month == '02' or month == '03':
            year = '2017'

        for day in t:

            if day < 10:
                day = '0%s' %day
            else:
                day = '%s' %day


            Date = int(year+month+day)

            nc_dir = '../../../Data/Weathermast_MEPS_Retrieval/%s/%s/weathermast_meps_retrieval_%s_12.nc' %(station,meps,Date)

            f[Date] = netCDF4.Dataset(nc_dir, 'r')
            if len(f[Date].variables) != 0:
#                print(Date, 'no valid data')
 #           else:
                if np.nansum(f[Date].variables['obs_precipitation_difference_2degC']) >= 0.25 \
                and np.nansum(f[Date].variables['meps_precipitation_difference_2degC']) >= 0.25 \
                and np.nansum(f[Date].variables['ret_precipitation_difference']) >= 0.25:
                    date = np.append(date, int(Date))
                   # print(Date)
                    obs_wind_dir[Date]         = f[Date].variables['obs_wind_direction'][:]
                    obs_wind_speed[Date]       = f[Date].variables['obs_wind_speed'][:]
                    obs_precip_diff[Date]      = f[Date].variables['obs_precipitation_difference'][:]
                    obs_precip_diff_2deg[Date] = f[Date].variables['obs_precipitation_difference_2degC'][:]
                    obs_west_idx[Date]         = f[Date].variables['obs_idx_westerly'][:].astype(int)
                    obs_east_idx[Date]         = f[Date].variables['obs_idx_easterly'][:].astype(int)

                    meps_wind_dir[meps][Date]        = f[Date].variables['meps_wind_direction'][:]
                    meps_wind_speed[meps][Date]      = f[Date].variables['meps_wind_speed'][:]
                    meps_precip_diff[meps][Date]     = f[Date].variables['meps_precipitation_difference'][:]
                    meps_precip_diff_2deg[meps][Date]= f[Date].variables['meps_precipitation_difference_2degC'][:]
                    meps_west_idx[meps][Date]        = f[Date].variables['meps_idx_westerly'][:]
                    meps_east_idx[meps][Date]        = f[Date].variables['meps_idx_easterly'][:]

                    ret_precip_diff[Date]      = f[Date].variables['ret_precipitation_difference'][:]

                    
                    ############################################
                    ## assign 24h sums
                    if len(obs_west_idx[Date]) == 0 and len(obs_east_idx[Date]) == 0:
                        obs_precip_total[Date] = np.nan
                        obs_precip_west[Date]  = np.nan
                        obs_precip_east[Date]  = np.nan
                        
                    ### observation == reference
                    obs_precip_total_2deg[Date] = np.nansum(obs_precip_diff_2deg[Date][obs_west_idx[Date],]) + \
                                                  np.nansum(obs_precip_diff_2deg[Date][obs_east_idx[Date],])
                    obs_precip_west_2deg[Date]  = np.nansum(obs_precip_diff_2deg[Date][obs_west_idx[Date],])
                    obs_precip_east_2deg[Date]  = np.nansum(obs_precip_diff_2deg[Date][obs_east_idx[Date],])

                    ### retrieval
                    ret_precip_total[Date] = np.nansum(ret_precip_diff[Date][obs_west_idx[Date],]) + \
                                             np.nansum(ret_precip_diff[Date][obs_east_idx[Date],])
                    ret_precip_west[Date]  = np.nansum(ret_precip_diff[Date][obs_west_idx[Date],])
                    ret_precip_east[Date]  = np.nansum(ret_precip_diff[Date][obs_east_idx[Date],])
                
                    ### MEPS
                    meps_precip_total_2deg[meps][Date] = np.nansum(meps_precip_diff_2deg[meps][Date][~np.isnan(meps_west_idx[meps][Date]),]) + \
                                                         np.nansum(meps_precip_diff_2deg[meps][Date][~np.isnan(meps_east_idx[meps][Date]),])
                    meps_precip_west_2deg[meps][Date] = np.nansum(meps_precip_diff_2deg[meps][Date][~np.isnan(meps_west_idx[meps][Date]),])
                    meps_precip_east_2deg[meps][Date] = np.nansum(meps_precip_diff_2deg[meps][Date][~np.isnan(meps_east_idx[meps][Date]),])
            try:

                print('{:} (west, east, total); '
                          'obs: {:.2f}, {:.2f}, {:.2f}; '
                          'ret: {:.2f}, {:.2f}, {:.2f}; \
                          MEPS - {:}: {:.2f}, {:.2f}, {:.2f}; '
                          'MEPS - {:}: {:.2f}, {:.2f}, {:.2f};'.format(Date,
                                                                      obs_precip_west_2deg[Date],
                                                                      obs_precip_east_2deg[Date],
                                                                      obs_precip_total_2deg[Date],
                                                                      ret_precip_west[Date],
                                                                      ret_precip_east[Date],
                                                                      ret_precip_total[Date],
                                                                      'CTRL',
                                                                      meps_precip_west_2deg['CTRL'][Date],
                                                                      meps_precip_east_2deg['CTRL'][Date],
                                                                      meps_precip_total_2deg['CTRL'][Date],
                                                                      'XCCR',
                                                                      meps_precip_west_2deg['XCCR'][Date],
                                                                      meps_precip_east_2deg['XCCR'][Date],
                                                                      meps_precip_total_2deg['XCCR'][Date]))
            except KeyError:
                continue
                        
                        
            f[Date].close()

            ############################################
       


20161201 (west, east, total); obs: 2.10, 0.00, 2.10; ret: 0.27, 0.00, 0.27;                           MEPS - CTRL: 0.52, 0.00, 0.52; MEPS - XCCR: 0.84, 0.00, 0.84;
20161206 (west, east, total); obs: 5.69, 1.99, 7.68; ret: 5.41, 2.06, 7.46;                           MEPS - CTRL: 0.43, 6.24, 6.67; MEPS - XCCR: 0.41, 7.61, 8.02;
20161208 (west, east, total); obs: 3.98, 0.00, 3.98; ret: 1.10, 0.00, 1.10;                           MEPS - CTRL: 7.11, 0.00, 7.11; MEPS - XCCR: 8.42, 0.00, 8.42;
20161209 (west, east, total); obs: 4.93, 2.15, 7.08; ret: 0.22, 1.72, 1.94;                           MEPS - CTRL: 3.98, 2.95, 6.93; MEPS - XCCR: 5.93, 4.47, 10.40;
20161210 (west, east, total); obs: 1.80, 0.00, 1.80; ret: 0.38, 0.00, 0.38;                           MEPS - CTRL: 6.58, 0.00, 6.58; MEPS - XCCR: 11.43, 0.00, 11.43;
20161212 (west, east, total); obs: 0.73, 0.12, 0.85; ret: 0.59, 0.09, 0.68;                           MEPS - CTRL: 0.77, 0.31, 1.08; MEPS - XCCR: 1.93, 0.00, 1.93;
20161215 (wes

# Get monthly sum of precipitation and calculate % difference

\begin{equation*}
\frac{RET - OBS}{OBS} * 100 = Diff [\%]
\end{equation*}

\begin{equation*}
\frac{MEPS - OBS}{OBS} * 100 = Diff [\%]
\end{equation*}


In [6]:
def calc_monthly_precip_sum(date, precip_total, precip_east, precip_west):
    
    # calculate monthly precip sum observation for <2degC, and valid days
    total = []
    total_east = []
    total_west  = []


    monthly_sum  = 0.0
    monthly_east_sum  = 0.0
    monthly_west_sum  = 0.0
    for Date in date:
        Date = int(Date)

        if Date == int(20161201) or Date == int(20170102) or Date == int(20170202):
            monthly_sum  = 0.0
            monthly_east_sum  = 0.0
            monthly_west_sum  = 0.0

        tot_val  = (precip_total[Date])
        monthly_sum  = np.sum([ monthly_sum ,tot_val ])

        east_val  = (precip_east[Date])
        monthly_east_sum  = np.sum([ monthly_east_sum , east_val ])

        west_val  = (precip_west[Date])
        monthly_west_sum  = np.sum([ monthly_west_sum , west_val ])

        if Date == int(20161231) or Date == int(20170128) or Date == int(20170206):
                     total.append( monthly_sum )
                     total_east.append( monthly_east_sum )
                     total_west.append( monthly_west_sum )

    
    return(np.asarray(total),  np.asarray(total_east),  np.asarray(total_west))

In [7]:
def calc_percentage_diff(obs_total, ret_total):
    diff_perc_monthly = (ret_total - obs_total)/obs_total * 100
    diff_perc_season  = (np.sum(ret_total) - np.sum(obs_total))/np.sum(obs_total) * 100
    
    return(diff_perc_monthly, diff_perc_season)

In [8]:
## DoFe observation
obs_total, obs_total_east, obs_total_west = calc_monthly_precip_sum(date, obs_precip_total_2deg, obs_precip_east_2deg, obs_precip_west_2deg)

## Retrieval
ret_total, ret_total_east, ret_total_west = calc_monthly_precip_sum(date, ret_precip_total, ret_precip_east, ret_precip_west)
# seasonal
ret_obs_monthly, ret_obs_season           = calc_percentage_diff(obs_total, ret_total)
ret_obs_monthly_east, ret_obs_season_east = calc_percentage_diff(obs_total_east, ret_total_east)
ret_obs_monthly_west, ret_obs_season_west = calc_percentage_diff(obs_total_west, ret_total_west)

## MEPS
meps_total = dict()
meps_total_east = dict()
meps_total_west = dict()
meps_obs_monthly = dict()
meps_obs_season  = dict()
meps_obs_monthly_east = dict()
meps_obs_season_east  = dict()
meps_obs_monthly_west = dict()
meps_obs_season_west  = dict()
for meps in meps_run:
    meps_total[meps], meps_total_east[meps], meps_total_west[meps] = calc_monthly_precip_sum(date, meps_precip_total_2deg[meps], meps_precip_east_2deg[meps], meps_precip_west_2deg[meps])
    
    meps_obs_monthly[meps], meps_obs_season[meps] = calc_percentage_diff(obs_total, meps_total[meps])
    meps_obs_monthly_east[meps], meps_obs_season_east[meps] = calc_percentage_diff(obs_total_east, meps_total_east[meps])
    meps_obs_monthly_west[meps], meps_obs_season_west[meps] = calc_percentage_diff(obs_total_west, meps_total_west[meps])

In [9]:
### create table for Retrieval Obs result
col_labels = ['Obs [mm]', 'Retrieved [mm]','Difference [%]', ]
row_labels = ['', 'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017',
              '', 
              '',  'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017',
              '',
              '',  'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017'
             ]
table_vals = [ ['', '' , '' ],
               ['{:.2f}'.format(obs_total[0]), '{:.2f}'.format(ret_total[0]), '{:.2f}'.format(ret_obs_monthly[0])],
               ['{:.2f}'.format(obs_total[1]), '{:.2f}'.format(ret_total[1]), '{:.2f}'.format(ret_obs_monthly[1])],
               ['{:.2f}'.format(obs_total[2]), '{:.2f}'.format(ret_total[2]), '{:.2f}'.format(ret_obs_monthly[2])],
               ['{:.2f}'.format(np.sum(obs_total)), '{:.2f}'.format(np.sum(ret_total)), '{:.2f}'.format(ret_obs_season)],
             #
               ['', '' , '' ],
               ['', '' , '' ],
               ['{:.2f}'.format(obs_total_west[0]), '{:.2f}'.format(ret_total_west[0]), '{:.2f}'.format(ret_obs_monthly_west[0])],
               ['{:.2f}'.format(obs_total_west[1]), '{:.2f}'.format(ret_total_west[1]), '{:.2f}'.format(ret_obs_monthly_west[1])],
               ['{:.2f}'.format(obs_total_west[2]), '{:.2f}'.format(ret_total_west[2]), '{:.2f}'.format(ret_obs_monthly_west[2])],
               ['{:.2f}'.format(np.sum(obs_total_west)), '{:.2f}'.format(np.sum(ret_total_west)), '{:.2f}'.format(ret_obs_season_west)],
             #
               ['', '' , '' ],
               ['', '' , '' ],
               ['{:.2f}'.format(obs_total_east[0]), '{:.2f}'.format(ret_total_east[0]), '{:.2f}'.format(ret_obs_monthly_east[0])],
               ['{:.2f}'.format(obs_total_east[1]), '{:.2f}'.format(ret_total_east[1]), '{:.2f}'.format(ret_obs_monthly_east[1])],
               ['{:.2f}'.format(obs_total_east[2]), '{:.2f}'.format(ret_total_east[2]), '{:.2f}'.format(ret_obs_monthly_east[2])],
               ['{:.2f}'.format(np.sum(obs_total_east)), '{:.2f}'.format(np.sum(ret_total_east)), '{:.2f}'.format(ret_obs_season_east)]
             ]

# Draw table
fig, axsm = plt.subplots(1, 1, #sharex = 'all', 
                         # sharey = 'all',
                         #figsize = [19.5,6]
                        )
the_table = axsm.table(cellText = table_vals,
                      rowLabels= row_labels,
                      colLabels= col_labels,
                      loc = 'center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(14)
the_table.scale(1.2,1.7)



# Removing ticks and spines enables you to get the figure only with table
axsm.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
axsm.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
for pos in ['right','top','bottom','left']:
        plt.gca().spines[pos].set_visible(False)
        
axsm.text(0.4, 1.41, 'Retrieval',
            verticalalignment='center', horizontalalignment='center',
            transform=axsm.transAxes,
            color='black', #fontsize=15, 
            fontweight='bold', bbox={'facecolor': 'grey', 'alpha': 0.5, 'pad': 5.5});
axsm.text(-0.32, 1.2, 'Total Precipitation',
          verticalalignment='center', horizontalalignment='center',
            transform=axsm.transAxes,
            color='black', #fontsize=15, 
            #fontweight='bold', 
          bbox={'facecolor': 'k', 'alpha': 0.5, #'pad': 5.5
                                    });

axsm.text(-0.4, 0.64, 'Westerly',
          verticalalignment='center', horizontalalignment='center',
            transform=axsm.transAxes,
            color='black', #fontsize=15, 
            #fontweight='bold', 
          bbox={'facecolor': 'b', 'alpha': 0.5, #'pad': 5.5
                                    });
axsm.text(-0.4, 0.08, 'Easterly',
          verticalalignment='center', horizontalalignment='center',
            transform=axsm.transAxes,
            color='black', #fontsize=15, 
            #fontweight='bold', 
          bbox={'facecolor': 'r', 'alpha': 0.5, #'pad': 5.5
                                    });



if savefig == 1:
        fig_name = 'winter_16_17_percent_diff_ret'+'.'+form
        plt.savefig('%s/%s' % (figdir, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/%s' %(figdir,fig_name))
        plt.close()

plot saved: ../../../Figures/Weathermast_MEPS_Retrieval/Haukeliseter/CTRL_XCCR//winter_16_17_percent_diff_ret.png


In [10]:
### create table for Retrieval Obs result
col_labels = ['Obs [mm]', 'CTRL [mm]','Difference [%]', 'XCCR [mm]','Difference [%]',]
row_labels = ['', 'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017',
              '', 
              '',  'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017',
              '',
              '',  'Dec 2016', 'Jan 2017', 'Feb 2017',  'Winter 2016-2017'
             ]
table_vals = [ ['', '' , '' , '' , '' ],
               ['{:.2f}'.format(obs_total[0]), '{:.2f}'.format(meps_total['CTRL'][0]), '{:.2f}'.format(meps_obs_monthly['CTRL'][0]), '{:.2f}'.format(meps_total['XCCR'][0]), '{:.2f}'.format(meps_obs_monthly['XCCR'][0])],
               ['{:.2f}'.format(obs_total[1]), '{:.2f}'.format(meps_total['CTRL'][1]), '{:.2f}'.format(meps_obs_monthly['CTRL'][1]), '{:.2f}'.format(meps_total['XCCR'][1]), '{:.2f}'.format(meps_obs_monthly['XCCR'][1])],
               ['{:.2f}'.format(obs_total[2]), '{:.2f}'.format(meps_total['CTRL'][2]), '{:.2f}'.format(meps_obs_monthly['CTRL'][2]), '{:.2f}'.format(meps_total['XCCR'][2]), '{:.2f}'.format(meps_obs_monthly['XCCR'][2])],
               ['{:.2f}'.format(np.sum(obs_total)), '{:.2f}'.format(np.sum(meps_total['CTRL'])), '{:.2f}'.format(meps_obs_season['CTRL']), '{:.2f}'.format(np.sum(meps_total['XCCR'])), '{:.2f}'.format(meps_obs_season['XCCR'])],
              # 
               ['', '' , '' , '' , '' ],
               ['', '' , '' , '' , '' ],
               ['{:.2f}'.format(obs_total_west[0]), '{:.2f}'.format(meps_total_west['CTRL'][0]), '{:.2f}'.format(meps_obs_monthly_west['CTRL'][0]), 
                                                    '{:.2f}'.format(meps_total_west['XCCR'][0]), '{:.2f}'.format(meps_obs_monthly_west['XCCR'][0])],
               ['{:.2f}'.format(obs_total_west[1]), '{:.2f}'.format(meps_total_west['CTRL'][1]), '{:.2f}'.format(meps_obs_monthly_west['CTRL'][1]), 
                                                    '{:.2f}'.format(meps_total_west['XCCR'][1]), '{:.2f}'.format(meps_obs_monthly_west['XCCR'][1])],
               ['{:.2f}'.format(obs_total_west[2]), '{:.2f}'.format(meps_total_west['CTRL'][2]), '{:.2f}'.format(meps_obs_monthly_west['CTRL'][2]), 
                                                    '{:.2f}'.format(meps_total_west['XCCR'][2]), '{:.2f}'.format(meps_obs_monthly_west['XCCR'][2])],
               ['{:.2f}'.format(np.sum(obs_total_west)), '{:.2f}'.format(np.sum(meps_total_west['CTRL'])), '{:.2f}'.format(meps_obs_season_west['CTRL']), 
                                                         '{:.2f}'.format(np.sum(meps_total_west['XCCR'])), '{:.2f}'.format(meps_obs_season_west['XCCR'])],
               ['', '' , '' , '' , '' ],
               ['', '' , '' , '' , '' ],
               ['{:.2f}'.format(obs_total_east[0]), '{:.2f}'.format(meps_total_east['CTRL'][0]), '{:.2f}'.format(meps_obs_monthly_east['CTRL'][0]), 
                                                    '{:.2f}'.format(meps_total_east['XCCR'][0]), '{:.2f}'.format(meps_obs_monthly_east['XCCR'][0])],
               ['{:.2f}'.format(obs_total_east[1]), '{:.2f}'.format(meps_total_east['CTRL'][1]), '{:.2f}'.format(meps_obs_monthly_east['CTRL'][1]), 
                                                    '{:.2f}'.format(meps_total_east['XCCR'][1]), '{:.2f}'.format(meps_obs_monthly_east['XCCR'][1])],
               ['{:.2f}'.format(obs_total_east[2]), '{:.2f}'.format(meps_total_east['CTRL'][2]), '{:.2f}'.format(meps_obs_monthly_east['CTRL'][2]), 
                                                    '{:.2f}'.format(meps_total_east['XCCR'][2]), '{:.2f}'.format(meps_obs_monthly_east['XCCR'][2])],
               ['{:.2f}'.format(np.sum(obs_total_east)), '{:.2f}'.format(np.sum(meps_total_east['CTRL'])), '{:.2f}'.format(meps_obs_season_east['CTRL']), 
                                                         '{:.2f}'.format(np.sum(meps_total_east['XCCR'])), '{:.2f}'.format(meps_obs_season_east['XCCR'])],
               
             ]


# Draw table
fig, axsm = plt.subplots(1, 1, #sharex = 'all', 
                         # sharey = 'all',
                         #figsize = [19.5,6]
                        )
the_table = axsm.table(cellText = table_vals,
                      rowLabels= row_labels,
                      colLabels= col_labels,
                      loc = 'center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(14)
the_table.scale(1.8,1.7)




# Removing ticks and spines enables you to get the figure only with table
axsm.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
axsm.tick_params(axis='y', which='both', right=False, left=False, labelleft=False)
for pos in ['right','top','bottom','left']:
        plt.gca().spines[pos].set_visible(False)
        
axsm.text(0.4, 1.41, 'MEPS',
            verticalalignment='center', horizontalalignment='center',
            transform=axsm.transAxes,
            color='black', #fontsize=15, 
            fontweight='bold', bbox={'facecolor': 'grey', 'alpha': 0.5, 'pad': 5.5});
axsm.text(-0.63, 1.2, 'Total Precipitation',
          verticalalignment='center', horizontalalignment='center',
            transform=axsm.transAxes,
            color='black', #fontsize=15, 
            #fontweight='bold', 
          bbox={'facecolor': 'k', 'alpha': 0.5, #'pad': 5.5
                                    });

axsm.text(-0.71, 0.64, 'Westerly',
          verticalalignment='center', horizontalalignment='center',
            transform=axsm.transAxes,
            color='black', #fontsize=15, 
            #fontweight='bold', 
          bbox={'facecolor': 'b', 'alpha': 0.5, #'pad': 5.5
                                    });
axsm.text(-0.71, 0.08, 'Easterly',
          verticalalignment='center', horizontalalignment='center',
            transform=axsm.transAxes,
            color='black', #fontsize=15, 
            #fontweight='bold', 
          bbox={'facecolor': 'r', 'alpha': 0.5, #'pad': 5.5
                                    });
plt.tight_layout()

if savefig == 1:
        fig_name = 'winter_16_17_percent_diff_MEPS'+'.'+form
        plt.savefig('%s/%s' % (figdir, fig_name), format = form, bbox_inches='tight', transparent=True)
        print('plot saved: %s/%s' %(figdir,fig_name))
        plt.close()

plot saved: ../../../Figures/Weathermast_MEPS_Retrieval/Haukeliseter/CTRL_XCCR//winter_16_17_percent_diff_MEPS.png


# Seperate precipiation into wind direction and wind speed classes

- easterlies: 22.5 - 202.5
- westerlies: 202.5 - 22.5$^{o}C$

WS: [0 - 4), [4 - 8), [8 - 12), [12 - 16), [16 - 20), [20 - inf) m s$^{-1}$

In [11]:
### double fence observation
obs_precip_west_0_4, obs_precip_east_0_4, \
obs_precip_west_4_8, obs_precip_east_4_8, \
obs_precip_west_8_12, obs_precip_east_8_12, \
obs_precip_west_12_16,obs_precip_east_12_16, \
obs_precip_west_16_20,obs_precip_east_16_20, \
obs_precip_west_20,obs_precip_east_20 =  fct.get_precip_amount_for_wind_speed(obs_precip_diff_2deg, obs_wind_dir, obs_west_idx, obs_east_idx, obs_wind_speed)

In [12]:
obs_precip_west_ws = [np.nansum(obs_precip_west_0_4[:,:],axis= 0),
                      np.nansum(obs_precip_west_4_8[:,:],axis= 0),
                      np.nansum(obs_precip_west_8_12[:,:],axis= 0),
                      np.nansum(obs_precip_west_12_16[:,:],axis= 0),
                      np.nansum(obs_precip_west_16_20[:,:],axis= 0),
                      np.nansum(obs_precip_west_20[:,:],axis= 0)]
obs_precip_west_ws = np.array(obs_precip_west_ws)
obs_precip_west_ws[np.where(obs_precip_west_ws[:,:] == 0.0)] = np.nan

obs_precip_east_ws = [np.nansum(obs_precip_east_0_4[:,:],axis= 0),
                      np.nansum(obs_precip_east_4_8[:,:],axis= 0),
                      np.nansum(obs_precip_east_8_12[:,:],axis= 0),
                      np.nansum(obs_precip_east_12_16[:,:],axis= 0),
                      np.nansum(obs_precip_east_16_20[:,:],axis= 0),
                      np.nansum(obs_precip_east_20[:,:],axis= 0)]
obs_precip_east_ws = np.array(obs_precip_east_ws)
obs_precip_east_ws[np.where(obs_precip_east_ws[:,:] == 0.0)] = np.nan

In [13]:
### retrieved values
ret_precip_west_0_4, ret_precip_east_0_4, \
ret_precip_west_4_8, ret_precip_east_4_8, \
ret_precip_west_8_12, ret_precip_east_8_12, \
ret_precip_west_12_16,ret_precip_east_12_16, \
ret_precip_west_16_20,ret_precip_east_16_20, \
ret_precip_west_20,ret_precip_east_20 =  fct.get_precip_amount_for_wind_speed(ret_precip_diff, obs_wind_dir, obs_west_idx, obs_east_idx, obs_wind_speed)


In [14]:
ret_precip_west_ws = [np.nansum(ret_precip_west_0_4[:,:],axis= 0),
                      np.nansum(ret_precip_west_4_8[:,:],axis= 0),
                      np.nansum(ret_precip_west_8_12[:,:],axis= 0),
                      np.nansum(ret_precip_west_12_16[:,:],axis= 0),
                      np.nansum(ret_precip_west_16_20[:,:],axis= 0),
                      np.nansum(ret_precip_west_20[:,:],axis= 0)]
ret_precip_west_ws = np.array(ret_precip_west_ws)
ret_precip_west_ws[np.where(ret_precip_west_ws[:,:] == 0.0)] = np.nan

ret_precip_east_ws = [np.nansum(ret_precip_east_0_4[:,:],axis= 0),
                      np.nansum(ret_precip_east_4_8[:,:],axis= 0),
                      np.nansum(ret_precip_east_8_12[:,:],axis= 0),
                      np.nansum(ret_precip_east_12_16[:,:],axis= 0),
                      np.nansum(ret_precip_east_16_20[:,:],axis= 0),
                      np.nansum(ret_precip_east_20[:,:],axis= 0)]
ret_precip_east_ws = np.array(ret_precip_east_ws)
ret_precip_east_ws[np.where(ret_precip_east_ws[:,:] == 0.0)] = np.nan

## MEPS

In [15]:
meps_precip_west_ws = dict()
meps_precip_east_ws = dict()


for meps in meps_run:
    ### meps forecast
    meps_precip_west_0_4, meps_precip_east_0_4, \
    meps_precip_west_4_8, meps_precip_east_4_8, \
    meps_precip_west_8_12, meps_precip_east_8_12, \
    meps_precip_west_12_16,meps_precip_east_12_16, \
    meps_precip_west_16_20,meps_precip_east_16_20, \
    meps_precip_west_20,meps_precip_east_20 =  fct.get_precip_amount_for_wind_speed_MEPS(meps_precip_diff_2deg[meps], 
                                                                                               meps_wind_dir[meps], 
                                                                                               meps_west_idx[meps], 
                                                                                               meps_east_idx[meps], 
                                                                                               meps_wind_speed[meps])

    meps_precip_west_ws[meps] = [np.nansum(meps_precip_west_0_4[:,:],axis= 0),
                          np.nansum(meps_precip_west_4_8[:,:],axis= 0),
                          np.nansum(meps_precip_west_8_12[:,:],axis= 0),
                          np.nansum(meps_precip_west_12_16[:,:],axis= 0),
                          np.nansum(meps_precip_west_16_20[:,:],axis= 0),
                          np.nansum(meps_precip_west_20[:,:],axis= 0)]
    meps_precip_west_ws[meps] = np.array(meps_precip_west_ws[meps])
    meps_precip_west_ws[meps][np.where(meps_precip_west_ws[meps][:,:] == 0.0)] = np.nan

    #meps_precip_west_ws_mean = np.nanmean(meps_precip_west_ws[:,1:], axis = 1)

    meps_precip_east_ws[meps] = [np.nansum(meps_precip_east_0_4[:,:],axis= 0),
                          np.nansum(meps_precip_east_4_8[:,:],axis= 0),
                          np.nansum(meps_precip_east_8_12[:,:],axis= 0),
                          np.nansum(meps_precip_east_12_16[:,:],axis= 0),
                          np.nansum(meps_precip_east_16_20[:,:],axis= 0),
                          np.nansum(meps_precip_east_20[:,:],axis= 0)]
    meps_precip_east_ws[meps] = np.array(meps_precip_east_ws[meps])
    meps_precip_east_ws[meps][np.where(meps_precip_east_ws[meps][:,:] == 0.0)] = np.nan

    #meps_precip_east_ws_mean = np.nanmean(meps_precip_east_ws[:,1:], axis = 1)

In [16]:
def plt_precip_in_wind_category(axs, west, east, observation):
    
    bar_width = 0.35
    opacity = 0.4
    
    bar11 = axs.bar(np.arange(west.shape[0]) - bar_width/2,
                    west[:], width = bar_width,
                    align = 'center', alpha = opacity, color = 'b',
                    label = 'westerlies {:.1f}'.format(np.nansum(west[:])))
    bar12 = axs.bar(np.arange(east.shape[0]) + bar_width/2,
                    east[:], width = bar_width,
                    align = 'center', alpha = opacity, color = 'r',
                    label = 'westerlies {:.1f}'.format(np.nansum(east[:])))
    axs.text(0.4, 0.95, observation,
            verticalalignment='center', horizontalalignment='center',
            transform=axs.transAxes,
            color='black', fontsize=15, fontweight='bold', bbox={'facecolor': 'grey', 'alpha': 0.5, 'pad': 5.5})
    # Add counts above the two bar graphs
    for rect in bar11 + bar12:
        height = rect.get_height()
        if np.isnan(height):
            continue
        else:
            axs.text(rect.get_x() + rect.get_width()/2.0, height, '{:.1f}'.format(height), ha='center', va='bottom')
    axs.legend(bbox_to_anchor=(0,1.02,1,0.2), loc='lower left', mode='expand',fancybox=True, facecolor ='white')

    
    axs.set_xticklabels(['[0.0 : 4.0)', '[4.0 : 8.0)', '[8.0 : 12.0)', '[12.0 : 16.0)', '[16.0 : 20.0)', '[20.0 : inf)'], rotation=30)



In [17]:
fig, axsm = plt.subplots(1, 4, sharex = 'all', 
                          sharey = 'all',
                         figsize = [19.5,6])

axs = axsm.flatten()

for i in range(axs.shape[0]):
    axs[i].set_xlabel('Wind speed (m/s)')
axs[0].set_ylabel('precipitation (mm)')
axs[0].set_xticks(np.arange(obs_precip_west_ws.shape[0]))

plt_precip_in_wind_category(axs[0], obs_precip_west_ws[:,1],
                            obs_precip_east_ws[:,1], 'Double fence')

plt_precip_in_wind_category(axs[1], ret_precip_west_ws[:,1],
                            ret_precip_east_ws[:,1], 'Retrieval')

for i, meps in zip(np.arange(2,np.array(meps_run).shape[0]+2), meps_run):
    plt_precip_in_wind_category(axs[i], meps_precip_west_ws[meps][:,1],
                                meps_precip_east_ws[meps][:,1], 'MEPS - %s' %meps)

plt.tight_layout()
if savefig == 1:
    cF.createFolder('%s/' %(figdir))
    fig_name = 'winter_16_17_sfc_precip_WS_WD.'+form
    plt.savefig('%s/%s.%s' %(figdir, fig_name,form), format = form, bbox_inches='tight', transparent=True)
    print('plot saved: %s/%s' %(figdir, fig_name))
    plt.close()

plot saved: ../../../Figures/Weathermast_MEPS_Retrieval/Haukeliseter/CTRL_XCCR//winter_16_17_sfc_precip_WS_WD.png
